# Assignment 1 – Backprop

In [ ]:
#@title Library Imports [do not change]

import importlib 
!git clone https://www.github.com/rycolab/intro-nlp-f22.git
utils = importlib.import_module("intro-nlp-f22.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f22'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 6), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "2" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': '(x^2 - 1) * (y+2)', 'in_vars': {'x': 3.0, 'y': 2.0}, 'output': 32.0, 'derivative': {'x': 24.0, 'y': 8.0}}
[[['x', '^', 2], '-', 1], '*', ['y', '+', 2]] {'x': 3.0, 'y': 2.0}


In [ ]:
#@title ToDo1: Building
# import copy
class Builder():
    
    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)

        # self.infix = copy.deepcopy(infix)
        self.infix = infix
        self.graph = {}
        ## ToDO: implement and set self.graph
        self.graph = self.fun(infix = [self.infix], graph = self.graph, available_vars=self.avail_vars, level=0, in_vars=in_vars)
        self.parse_graph(self.graph)

    def fun(self, infix, graph, available_vars, level = 0, in_vars = None):
      # First time, graph is made up of only input variables and their values
      if level == 0:
          graph = {}
          for var in in_vars.keys():
              graph[var] = in_vars[var]

      # Recursive calls:
      for i, item in enumerate(infix):
          if type(item) is list:
              self.fun(item, graph, available_vars, level+1)
              var = available_vars.pop()
              graph[var] = item
              infix[i] = var

      # Return only when finished iterating over the outermost list
      if level == 0:
          return graph

    def parse(self, expression):
      if type(expression) == list and len(expression) == 2:
          return {'operands': [expression[1]], 'function': expression[0], 'value': None, 'backwards': 0}
      elif type(expression) == list and len(expression) == 3:
          return {'operands': [expression[0], expression[2]], 'function': expression[1], 'value': None, 'backwards': 0}
      elif type(expression) == int or type(expression) == float:
          return {'operands': None, 'function': None, 'value': expression, 'backwards': 0}

    def parse_graph(self, graph):
      for key in graph.keys():
          # print(key, graph[key])
          graph[key] = self.parse(graph[key])

if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.graph)

{'x': {'operands': None, 'function': None, 'value': 3.0, 'backwards': 0}, 'y': {'operands': None, 'function': None, 'value': 2.0, 'backwards': 0}, 'a': {'operands': ['x', 2], 'function': '^', 'value': None, 'backwards': 0}, 'b': {'operands': ['a', 1], 'function': '-', 'value': None, 'backwards': 0}, 'c': {'operands': ['y', 2], 'function': '+', 'value': None, 'backwards': 0}, 'd': {'operands': ['b', 'c'], 'function': '*', 'value': None, 'backwards': 0}}


In [ ]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        ## ToDO: implement
        return math.log(a)

    def df(self, a, b = None):
        ## ToDO: implement
        return [1/a]

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a/b

    def df(self, a, b):
        ## ToDO: implement
        return [1/b, -a/(b*b)]

class Add(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a+b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1,1]

class Sub(Operator):

    def f(self, a, b = None):
        ## ToDO: implement
        return a-b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1, -1]

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.sin(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [math.cos(a)]

class Cos(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.cos(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [-math.sin(a)]

In [ ]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self):
        ## ToDO: implement and set self.output

        # Go through variables in topological order
        for key in self.graph.keys():

            # If the variable isn't yet evaluated, evaluate it based on parent nodes
            if self.graph[key]['value'] is None:

                # Each variable has parent nodes (operands), and the function to apply on the parent nodes (fun_to_use)
                fun_to_use = self.graph[key]['function']
                operands = self.graph[key]['operands'].copy()
                
                # Find the values of the parent nodes needed to compute the value of the current variable
                num_operands = len(operands)
                if num_operands == 1:
                    if operands[0] in self.graph.keys():
                        operands[0] = self.graph[operands[0]]['value']
                elif num_operands == 2:
                    if operands[0] in self.graph.keys():
                        operands[0] = self.graph[operands[0]]['value']
                    if operands[1] in self.graph.keys():
                        operands[1] = self.graph[operands[1]]['value']

                computed_val = self.fn_map[fun_to_use].f(*operands)

                self.graph[key]['value'] = computed_val
      
        self.output = self.graph[list(self.graph.keys())[-1]]['value']

    ## backward execution____________________________

    def backward(self):
        ## ToDO: implement and set self.derivative
        self.derivative = {}
        
        # Go through all nodes and set all  backwards values to 0
        for key in self.graph.keys():
          self.graph[key]['backwards'] = 0

        # 1. Go through nodes in reverse topological order (starting from the back). Last variable has backwards value = 1.
        # 2. For all children of the current node, find calculate the product of the corresponding gradient multiplied by the current node value
        # 3. Add the value to the children node backwards_value
        # (No need for recursion - just do it iteratively)

        output_flag = 0

        for key in reversed(list(self.graph.keys())):
            # print('Current key: ', key)

            # Set the derivative of output wrt itself to 1 (backwards value)
            if not output_flag:
                self.graph[key]['backwards'] = 1
                output_flag = 1

            # Calculate the gradient of current node wrt its parent nodes
            fun_to_use = self.graph[key]['function']

            parent_values = []

            operands = self.graph[key]['operands']
            if operands is not None:
              for operand in operands:
                if type(operand) is str:
                  parent_values.append(self.graph[operand]['value'])
                elif operand is not None:
                  parent_values.append(operand)

            # print('Operands: ', operands)
            # print('Values of operands: ', parent_values)
            # print('Function to use: ', fun_to_use)
            
            if fun_to_use is not None:
              gradient = self.fn_map[fun_to_use].df(*parent_values)

            # print('Gradients: ', gradient)

            # Do the rest of the variables
            if self.graph[key]['operands'] is not None:
                for i, parent in enumerate(self.graph[key]['operands']):
                  if (parent is not None) and (type(parent) is str):  
                    self.graph[parent]['backwards'] += self.graph[key]['backwards']*gradient[i]

            # print('Backwards value: ', self.graph[key]['backwards'])
            # print('-----------------------')
        # Final output
        for input_var in self.in_vars:
          self.derivative[input_var] = self.graph[input_var]['backwards']
                    

if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

32.0
{'x': 24.0, 'y': 8.0}


In [ ]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)


0: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
SUCCESS output: 32.0
SUCCESS derivative: {'x': 24.0, 'y': 8.0}


In [ ]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
SUCCESS output: 1.0
SUCCESS derivative: {'x': 1.0, 'y': -1.0}

1: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
SUCCESS output: 11.39
SUCCESS derivative: {'x': 7.39, 'y': -2}

2: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
SUCCESS output: 32.0
SUCCESS derivative: {'x': 24.0, 'y': 8.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
SUCCESS output: 0.14
SUCCESS derivative: {'x': -3.96, 'y': -0.99, 'z': 1.99}
